In [1]:
import requests
import re
from bs4 import BeautifulSoup

In [2]:
ratings = {}
k = 28
home_advantage = 22
season_home = 0
season_away = 0
total_home = 0
total_away = 0

In [3]:
ratings['현대캐피탈'] = 1500
ratings['삼성화재'] = 1500
ratings['KB손해보험'] = 1500
ratings['한국전력'] = 1500
ratings['대한항공'] = 1500
ratings['상무'] = 1500
ratings['우리카드'] = 1500
ratings['OK금융그룹'] = 1500

In [4]:
for season in range(1,19):
    season_home = 0
    season_away = 0
    if season <10:
        season = "0"+str(season)
    for game_number in range(1,400):
        url = "https://www.kovo.co.kr/game/v-league/11141_game-summary.asp?season=0"+str(season)+"&g_part=201&r_round=1&g_num="+str(game_number)+"&"
        res = requests.get(url)
        print(url)
        soup = BeautifulSoup(res.text,"lxml")
        team_names = soup.find_all("span",attrs={"class":"team"}) #팀 이름을 리스트로 가져옴
        if team_names == []:
            for key in ratings:
                if ratings[key] != 0:
                    ratings[key] = ratings[key]+(1500-ratings[key])/2
            break
        results = soup.find_all("p",attrs={"class":re.compile("^result")}) #승패 결과를 리스트로 가져옴
        points = soup.find_all("p",attrs={"class":"num"}) #세트 스코어를 리스트로 가져옴
        if points == []:
            break
        if season == 18:
            if points[0].get_text() == points[1].get_text():
                break
        home_team_name = team_names[0].get_text()
        away_team_name = team_names[1].get_text()
        if home_team_name == "LG화재":
            home_team_name = "KB손해보험"
        if away_team_name == "LG화재":
            away_team_name = "KB손해보험"
        if home_team_name == "LIG":
            home_team_name = "KB손해보험"
        if away_team_name == "LIG":
            away_team_name = "KB손해보험"
        if home_team_name == "LIG손해보험":
            home_team_name = "KB손해보험"
        if away_team_name == "LIG손해보험":
            away_team_name = "KB손해보험"
        if home_team_name == "KEPCO45":
            home_team_name = "한국전력"
        if away_team_name == "KEPCO45":
            away_team_name = "한국전력"
        if home_team_name == "KEPCO":
            home_team_name = "한국전력"
        if away_team_name == "KEPCO":
            away_team_name = "한국전력"
        if home_team_name == "신협상무":
            home_team_name = "상무"
        if away_team_name == "신협상무":
            away_team_name = "상무"
        if home_team_name == "우리캐피탈":
            home_team_name = "우리카드"
        if away_team_name == "우리캐피탈":
            away_team_name = "우리카드"
        if home_team_name == "상무신협":
            home_team_name = "상무"
        if away_team_name == "상무신협":
            away_team_name = "상무"
        if home_team_name == "드림식스":
            home_team_name = "우리카드"
        if away_team_name == "드림식스":
            away_team_name = "우리카드"
        if home_team_name == "러시앤캐시":
            home_team_name = "OK금융그룹"
        if away_team_name == "러시앤캐시":
            away_team_name = "OK금융그룹"
        if home_team_name == "OK저축은행":
            home_team_name = "OK금융그룹"
        if away_team_name == "OK저축은행":
            away_team_name = "OK금융그룹"
        if home_team_name not in ratings:
            continue
        if away_team_name not in ratings:
            continue
        expected_score = 1/(1+10**((ratings[away_team_name]-ratings[home_team_name]-home_advantage)/400))
        if int(points[0].get_text()) > int(points[1].get_text()):
            ratings[home_team_name] = ratings[home_team_name]+k*(1-expected_score)
            ratings[away_team_name] = ratings[away_team_name]-k*(1-expected_score)
            season_home+=1
            total_home+=1
        else:
            ratings[home_team_name] = ratings[home_team_name]+k*(0-expected_score)
            ratings[away_team_name] = ratings[away_team_name]-k*(0-expected_score)
            season_away+=1
            total_away+=1
    print(season_home,season_away,season_home/(season_home+season_away)*100)

https://www.kovo.co.kr/game/v-league/11141_game-summary.asp?season=001&g_part=201&r_round=1&g_num=1&
https://www.kovo.co.kr/game/v-league/11141_game-summary.asp?season=001&g_part=201&r_round=1&g_num=2&
https://www.kovo.co.kr/game/v-league/11141_game-summary.asp?season=001&g_part=201&r_round=1&g_num=3&
https://www.kovo.co.kr/game/v-league/11141_game-summary.asp?season=001&g_part=201&r_round=1&g_num=4&
https://www.kovo.co.kr/game/v-league/11141_game-summary.asp?season=001&g_part=201&r_round=1&g_num=5&
https://www.kovo.co.kr/game/v-league/11141_game-summary.asp?season=001&g_part=201&r_round=1&g_num=6&
https://www.kovo.co.kr/game/v-league/11141_game-summary.asp?season=001&g_part=201&r_round=1&g_num=7&
https://www.kovo.co.kr/game/v-league/11141_game-summary.asp?season=001&g_part=201&r_round=1&g_num=8&
https://www.kovo.co.kr/game/v-league/11141_game-summary.asp?season=001&g_part=201&r_round=1&g_num=9&
https://www.kovo.co.kr/game/v-league/11141_game-summary.asp?season=001&g_part=201&r_round=1

In [5]:
sorted_dict = sorted(ratings.items(), key = lambda item: item[0], reverse = True)
print(sorted_dict)

[('현대캐피탈', 1460.6982396068838), ('한국전력', 1512.3841130734322), ('우리카드', 1531.4951402877016), ('상무', 1499.744901817001), ('삼성화재', 1390.5607271652555), ('대한항공', 1576.8818597989123), ('OK금융그룹', 1496.0199874006348), ('KB손해보험', 1532.2150308501787)]


In [6]:
def xwin_rate(A,B):
    return 1/(1+10**((B-A-22)/400))

In [8]:
xwin_rate(ratings['현대캐피탈'], ratings['한국전력'])

0.45738225340584326

In [9]:
1/2.55

0.3921568627450981